In [14]:
import requests
import pandas as pd
import hopsworks

conn = hopsworks.connection()
HEILBONN_API_KEY = conn.get_secrets_api().get_secret("HEILBONN_API_KEY").value
    
# Define the API URL and API key
url = "https://34.149.159.161.nip.io/bildungscampus/iotplatform/mensaoccupancy/v1/authGroup/mensa_occupancy_devices/entityId?page=0"

# Set up headers with API key
headers = {
    'x-apikey': HEILBONN_API_KEY
}

# Send GET request to the API
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    
    # Initialize lists to store data
    location_names = []
    occupancies = []
    occupancy_ts = []
    capacities = []

    # Extract the necessary fields from the JSON response
    for entity in data['entities']:
        location_names.append(entity['SERVER_ATTRIBUTE']['locationName']['value'])
        occupancies.append(entity['TIME_SERIES']['occupancy']['value'])
        occupancy_ts.append(entity['TIME_SERIES']['occupancy']['ts'])
        capacities.append(entity['TIME_SERIES']['capacity']['value'])

##### We will not use these attributes in the simple example AI system
#         occupancy_relatives.append(entity['TIME_SERIES']['occupancyRelative']['value'])
#         occupancy_relative_ts.append(entity['TIME_SERIES']['occupancyRelative']['ts'])
#         location_ts.append(entity['SERVER_ATTRIBUTE']['locationName']['ts'])
#         out_counts.append(entity['TIME_SERIES']['outCount']['value'])
#         out_count_ts.append(entity['TIME_SERIES']['outCount']['ts'])
#         in_counts.append(entity['TIME_SERIES']['inCount']['value'])
#         in_count_ts.append(entity['TIME_SERIES']['inCount']['ts'])        
#         capacity_ts.append(entity['TIME_SERIES']['capacity']['ts'])

    # Create a DataFrame from the lists
    df = pd.DataFrame({
        'building': location_names,
        'y': occupancies,
        'ts': occupancy_ts,
        'capacity': capacities,
    })

    # Convert UNIX timestamps to pandas datetime
    df['ds'] = pd.to_datetime(df['ts'], unit='ms')
    # Convert pandas datetime to string
    df['ts'] = df['ds'].dt.strftime('%d.%m.%y %H:%M')
    # Convert str to double (errors become NaN)
    df['y'] = df['y'].astype(float)
    df['capacity'] = df['capacity'].astype(float)
    df['city'] = "Heilbronn"
    print(df)
else:
    print(f"Failed to retrieve data: {response.status_code} - {response.text}")


Connected. Call `.close()` to terminate connection gracefully.
           building    y              ts  capacity                      ds  \
0  Mensa Gebäude U8  0.0  18.08.24 19:54     500.0 2024-08-18 19:54:38.791   

        city  
0  Heilbronn  


In [15]:
import hopsworks
fs = hopsworks.login().get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/16503
Connected. Call `.close()` to terminate connection gracefully.


In [16]:
fg = fs.get_feature_group(name="mensa", version=1)

In [17]:
fg.insert(df)

Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: mensa_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/16503/jobs/named/mensa_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f0bc7791660>, None)